In [ ]:
import pandas as pd

In [ ]:
usage = pd.read_excel('/Nazrin/MO14-Round-1-Dealing-With-Data-Workbook.xlsx', sheet_name='Usage', header=None)

In [ ]:
usage

,0
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [ ]:
# removing underscores and spaces
usage[0] = usage[0].replace({'_':''}, regex=True).str.replace(r'\s{0,}', '') #2
#usage[0] = usage[0].apply(lambda x: x.strip())

In [ ]:
# extracting hours
t = usage[0].str.extract(r'(1[012]|[1-9])?(?i)(am|pm)') 
t['hour'] = t[[0, 1]].agg(''.join, axis=1)
t

,0,1,hour
0,3,PM,3PM
1,5,AM,5AM
2,8,PM,8PM
3,6,PM,6PM
4,1,AM,1AM
...,...,...,...
8755,1,AM,1AM
8756,6,AM,6AM
8757,2,AM,2AM
8758,9,PM,9PM


In [ ]:
#extracting month, day of the week, kwh, and subtracting them from the original raw line to get the days of the month
usage['month'] = usage[0].str.extract(r'((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)')
usage['hour'] = t['hour']
usage['w_day'] = usage[0].str.extract(r'(?i)(mon|tue|wed|thu|fri|sat|sun)')
usage['kwh'] = usage[0].apply(lambda x: x.split("2014")[1])
usage['remain'] = [a.replace(b, '').replace(c, '').replace(d, '').strip() for a, b, c, d in zip(usage[0], usage['kwh'], usage['hour'], usage['month'])]
usage

,0,month,hour,w_day,kwh,remain
0,3PMMon24th-Mar-20140.384kwh,Mar,3PM,Mon,0.384kwh,Mon24th--2014
1,5AM15-Aug-20141.201kwh,Aug,5AM,NaN,1.201kwh,15--2014
2,8PMThu20-Mar-20141.523kwh,Mar,8PM,Thu,1.523kwh,Thu20--2014
3,6PM23rd-Apr-20140.424kwh,Apr,6PM,NaN,0.424kwh,23rd--2014
4,1AMFriday19th-Dec-20140.209kwh,Dec,1AM,Fri,0.209kwh,Friday19th--2014
...,...,...,...,...,...,...
8755,1AMFri07th-Nov-20140.084kwh,Nov,1AM,Fri,0.084kwh,Fri07th--2014
8756,6AM20-May-20141.027kwh,May,6AM,NaN,1.027kwh,20--2014
8757,2AMTuesday8th-Apr-20140.052kwh,Apr,2AM,Tue,0.052kwh,Tuesday8th--2014
8758,9PM27th-Jan-20141.428kwh,Jan,9PM,NaN,1.428kwh,27th--2014


In [ ]:
#extracting days of months from ramain column
usage['day'] = usage['remain'].str.extract(r'([1,2,3][0-9]|[1-9])')
usage.drop(columns=[0, 'remain'], inplace=True)
usage

,month,hour,w_day,kwh,day
0,Mar,3PM,Mon,0.384kwh,24
1,Aug,5AM,NaN,1.201kwh,15
2,Mar,8PM,Thu,1.523kwh,20
3,Apr,6PM,NaN,0.424kwh,23
4,Dec,1AM,Fri,0.209kwh,19
...,...,...,...,...,...
8755,Nov,1AM,Fri,0.084kwh,7
8756,May,6AM,NaN,1.027kwh,20
8757,Apr,2AM,Tue,0.052kwh,8
8758,Jan,9PM,NaN,1.428kwh,27


In [ ]:
# removing 'kwh' string from kwh column, and converting its type to float
# converting type of day to int
usage['kwh'] = usage['kwh'].replace({'kwh':''}, regex=True).astype(float)
usage['day'] = usage['day'].astype(int)

In [ ]:
# sorting dataframe based on month and day columns
# na_position=last is used to fill missing values with ffil
usage.sort_values(by = ['month', 'day'], ascending=[True, True], inplace=True, 
               kind='quicksort', na_position='last')
usage

,month,hour,w_day,kwh,day
13,Apr,8PM,Tue,0.557,1
118,Apr,7PM,NaN,0.562,1
601,Apr,1PM,NaN,0.115,1
762,Apr,4AM,NaN,0.053,1
1047,Apr,2PM,Tue,0.143,1
...,...,...,...,...,...
6726,Sep,7AM,Tue,0.737,30
6756,Sep,9AM,NaN,0.216,30
8050,Sep,10AM,Tue,0.224,30
8351,Sep,7PM,Tue,1.462,30


In [ ]:
usage.isnull().sum()

month       0
hour        0
w_day    4301
kwh         0
day         0
dtype: int64

In [ ]:
usage.ffill(inplace=True)

In [ ]:
usage

,month,hour,w_day,kwh,day
13,Apr,8PM,Tue,0.557,1
118,Apr,7PM,Tue,0.562,1
601,Apr,1PM,Tue,0.115,1
762,Apr,4AM,Tue,0.053,1
1047,Apr,2PM,Tue,0.143,1
...,...,...,...,...,...
6726,Sep,7AM,Tue,0.737,30
6756,Sep,9AM,Tue,0.216,30
8050,Sep,10AM,Tue,0.224,30
8351,Sep,7PM,Tue,1.462,30


In [ ]:
usage.to_excel(r'/content/drive/MyDrive/usage.xlsx', index = False)

**The solutions of the tasks are in the excel file**